<a href="https://colab.research.google.com/github/envirodatascience/final-project-wildfire/blob/main/Merged%20Draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wildfires in the United States

This project examines the landscape of wildfires in the US from 1984-2024. We aim to answer the following questions using data from the US Geological Service that monitors trends in burn severity over this time period:

1) How has wildfire frequency changed over time in the US?

2) How has wildfire severity changed over time in the US?

3) Is prescribed burning an effective policy for mitigating fires?

First We Prepare our Data

In [1]:
# import standard packages
import pandas as pd
import numpy as np

#import plotting packages
import geopandas as gpd
from plotnine import *
import plotnine

#import stats packages
import scipy.stats as stats
import statsmodels.api as sm

#import animation packages
import matplotlib.pyplot as plt
import io
import os
from PIL import Image as PILImage
from IPython.display import display, Image
import imageio


# Upload Data
We are pulling data from The Monitoring Trends in Burn Severity (MTBS) Program provided by the USGS and USDA Forest Service

The data provides info regarding all currently inventoried fires occurring between calendar year 1984 and 2024 for CONUS, Alaska, Hawaii, and Puerto Rico.

In [2]:
# mount to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#upload fire severity data
! unzip /content/drive/MyDrive/mtbs_perims_DD.zip #unpacks zipped folder


unzip:  cannot find or open /content/drive/MyDrive/mtbs_perims_DD.zip, /content/drive/MyDrive/mtbs_perims_DD.zip.zip or /content/drive/MyDrive/mtbs_perims_DD.zip.ZIP.


In [4]:
shape = "mtbs_perims_DD.shp"
df = gpd.read_file(shape) #reads in shape file

DataSourceError: mtbs_perims_DD.shp: No such file or directory

#Orienting and Data Cleaning

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
#create state column from event ID
df['State'] = df['Event_ID'].str[:2]
df['State'].unique()

In [ ]:
df_state = df.pop('State')
df.insert(0,'State',df_state)
df.head()

In [ ]:
df['Incid_Type'].unique()

In [ ]:
# filter out columns
df_columns = df.drop(columns=['Event_ID', 'irwinID', 'Map_ID', 'Map_Prog', 'Asmnt_Type', 'Pre_ID', 'Post_ID', 'Perim_ID', 'Comment', 'dNBR_stdDv'])
df_columns.head()

In [ ]:
#check for missing/null values
df_columns['NoData_T'].unique()


In [ ]:
# df_columns['NoData_T'].value_counts()

#Wildfire Dynamics over Time

We want to investigate how fire size, severity, and frequency change over time.

In [ ]:
# filter for wildfires
df_wildfires = df_columns[df_columns['Incid_Type'] == 'Wildfire']

In [ ]:
# bucket by year
df_wildfires['Ig_Date'] = pd.to_datetime(df_wildfires['Ig_Date'])
df_wildfires['Year'] = df_wildfires['Ig_Date'].dt.year
df_wildfires_years = df_wildfires.pivot_table(index = 'Year', values = 'Incid_Type', aggfunc = 'count')
df_wildfires_years = df_wildfires_years.rename(columns = {'Incid_Type': 'Wildfires'})
df_wildfires_years = df_wildfires_years.reset_index()
df_wildfires_years.head()

Wildfire Frequency over Time

In [ ]:
# fire frequency over time
(
    ggplot(df_wildfires_years, aes(x = 'Year', y='Incid_Type'))
       + geom_bar(stat = 'identity', fill = 'blue')
       + geom_smooth(method='lm')
       + xlab("Year")
       + ylab("Wildfire Occurrences")
       + ggtitle("Wildfire Frequency Over Time")
       + scale_x_continuous(breaks = [1984, 1990, 1995, 2000, 2005, 2010, 2015, 2020, 2024])
       + theme_classic()

)

In [ ]:
# year of highest fire frequency
df_wildfires_years.sort_values(by = 'Incid_Type', ascending = False).head(1)

The graph shows that the frequency of fires has trended upward over time, with a peak in 2011.

Wildfire Size over Time

In [ ]:
# aggregate burned acres per year
df_wildfires_acres = df_wildfires.pivot_table(index = 'Year', values = 'BurnBndAc', aggfunc = 'sum')
df_wildfires_acres['BurnBndAc'] = df_wildfires_acres['BurnBndAc'].astype(int)
df_wildfires_acres['BurnBndAc'] = df_wildfires_acres['BurnBndAc'] / 1000000
df_wildfires_acres = df_wildfires_acres.reset_index()
df_wildfires_acres.head()

In [ ]:
# fire size over time
(
    ggplot(df_wildfires_acres, aes(x = 'Year', y='BurnBndAc'))
       + geom_bar(stat = 'identity', fill = 'red')
       + geom_smooth(method='lm')
       + xlab("Year")
       + ylab("Burned Area (Millions of Acres)")
       + ggtitle("Wildfire Size Over Time")
       + scale_x_continuous(breaks = [1984, 1990, 1995, 2000, 2005, 2010, 2015, 2020, 2024])
       + scale_y_continuous()
       + theme_classic()
)

In [ ]:
# check year of worst fires by acres burned
df_wildfires_acres.sort_values(by = 'BurnBndAc', ascending = False).head(1)

As the graph shows, the acres burned by wildfires has increased over time, with 2020 as the worst year for land affected by wildfires.  

# Wildfire Severity over Time

We measure wildfire severity by dNBR.

In [ ]:
#check dNBR details
df_wildfires['dNBR_offst'].describe()

In [ ]:
# aggregate dNBR per year
df_wildfires_dNBR = df_wildfires.pivot_table(index = 'Year', values = 'dNBR_offst', aggfunc = 'mean')
df_wildfires_dNBR = df_wildfires_dNBR.reset_index()
df_wildfires_dNBR.head()

In [ ]:
# dNBR over time
(
    ggplot(df_wildfires_dNBR, aes(x = 'Year', y='dNBR_offst'))
    + geom_bar(stat = 'identity', fill = 'green')
    + geom_smooth(method='lm')
    + theme_classic()
    + xlab("Year")
    + ylab("dNBR")
    + ggtitle("Wildfire Severity Over Time")
    + scale_x_continuous(breaks = [1984, 1990, 1995, 2000, 2005, 2010, 2015, 2020, 2024])
)

#Prescribed Burns vs Wildfires over Time

How does the occurrence of prescribed burns relate to wildfires?

In [ ]:
df_columns['Incid_Type'].unique()

In [ ]:
# filter for prescribed burns
df_prescribed = df_columns[df_columns['Incid_Type'] == 'Prescribed Fire']
df_prescribed.head()

In [ ]:
# prescribed fires by year

df_prescribed['Ig_Date'] = pd.to_datetime(df_prescribed['Ig_Date'])
df_prescribed['Year'] = df_prescribed['Ig_Date'].dt.year
df_prescribed_years = df_prescribed.pivot_table(index = 'Year', values = 'Incid_Type', aggfunc = 'count')
df_prescribed_years = df_prescribed_years.rename(columns = {'Incid_Type': 'Prescribed Fires'})
df_prescribed_years = df_prescribed_years.reset_index()
df_prescribed_years.head()


In [ ]:
# prescribed fire count over time
(
    ggplot(df_prescribed_years, aes(x = 'Year', y='Incid_Type'))
       + geom_bar(stat = 'identity', fill = 'orange')
       + geom_smooth(method='lm')
       + xlab("Year")
       + ylab("Prescribed Fire Occurrences")
       + ggtitle("Prescribed Fire Frequency Over Time")
       + scale_x_continuous(breaks = [1984, 1990, 1995, 2000, 2005, 2010, 2015, 2020, 2024])
       + theme_classic()
)

Now we want to look at how prescribed and wildfire trends relate over time.

In [ ]:
#merge frequency dataframes
df_freq_merged = pd.merge(df_wildfires_years, df_prescribed_years, on = 'Year')
df_freq_merged.head()

In [ ]:
# melt frequency
df_freq_melted = pd.melt(df_freq_merged, id_vars=['Year'], value_vars=['Wildfires', 'Prescribed Fires'], var_name='series', value_name='value')
df_freq_melted.head()

In [ ]:
# graph stacked bar chart of prescribed vs wildfires over time
(
    ggplot(df_freq_melted, aes(x = 'Year', y = 'value', fill = 'series'))
    + geom_bar(stat = 'identity')
    + xlab("Year")
    + ylab("Fire Occurrences")
    + labs(fill = 'Fire Type')
    + ggtitle("Prescribed Fire vs Wildfire Frequency Over Time")
    + scale_x_continuous(breaks = [1984, 1990, 1995, 2000, 2005, 2010, 2015, 2020, 2024])
    + theme_classic()
)


In [ ]:
# line graph of prescribed vs wildfire frequency over time
(
    ggplot(df_freq_melted, aes(x = 'Year', y = 'value', color = 'series'))
    + geom_line()
    + xlab("Year")
    + ylab("Fire Occurrences")
    + labs(color = 'Fire Type')
    + ggtitle("Prescribed Fire vs Wildfire Frequency Over Time")
    + scale_x_continuous(breaks = [1984, 1990, 1995, 2000, 2005, 2010, 2015, 2020, 2024])
    + theme_classic()
)

In [ ]:
# prescribed fires burn area
df_prescribed_acres = df_prescribed.pivot_table(index = 'Year', values = 'BurnBndAc', aggfunc = 'sum')
df_prescribed_acres['BurnBndAc'] = df_prescribed_acres['BurnBndAc'] / 1000000
df_prescribed_acres = df_prescribed_acres.reset_index()
df_prescribed_acres.head()

In [ ]:
# prescribed fires acres burned over time
(
    ggplot(df_prescribed_acres, aes(x = 'Year', y='BurnBndAc'))
    + geom_bar(stat = 'identity', fill = 'orange')
    + geom_smooth(method='lm')
    + xlab("Year")
    + ylab("Burned Area (Millions of Acres)")
    + ggtitle("Prescribed Fire Size Over Time")
    + scale_x_continuous(breaks = [1984, 1990, 1995, 2000, 2005, 2010, 2015, 2020, 2024])
    + scale_y_continuous()
    + theme_classic()
)

In [ ]:
# merge acres
df_acres_merged = pd.merge(df_wildfires_acres, df_prescribed_acres, on = 'Year')
df_acres_merged = df_acres_merged.rename(columns = {'BurnBndAc_x': 'Wildfires', 'BurnBndAc_y': 'Prescribed Fires'})
df_acres_merged.head()

In [ ]:
# melt acres
df_acres_melted = pd.melt(df_acres_merged, id_vars=['Year'], value_vars=['Wildfires', 'Prescribed Fires'], var_name='series', value_name='value')
df_acres_melted.head()

In [ ]:
# prescribed acres burned vs wildfire acres burned over time
(
    ggplot(df_acres_melted, aes(x = 'Year', y = 'value', fill = 'series'))
    + geom_bar(stat = 'identity')
    + xlab("Year")
    + ylab("Area Burned (Millions of Acres)")
    + labs(fill = 'Fire Type')
    + ggtitle("Prescribed Fire vs Wildfire Size Over Time")
    + scale_x_continuous(breaks = [1984, 1990, 1995, 2000, 2005, 2010, 2015, 2020, 2024])
    + theme_classic()
)


### State Analysis
We will take a deeper look into the three states that had the most fires during the study period: CA, FL, [___].

California

In [ ]:
#load in state boundary data
! unzip /content/drive/MyDrive/cb_2018_us_state_500k.zip #unpacks zipped folder

In [ ]:
shp = "cb_2018_us_state_500k.shp"
df_states = gpd.read_file(shp) #reads in shape file
df_states.head()

In [ ]:
#call california boundary
california = df_states[df_states['NAME'] == 'California']

In [ ]:
#Zoom in on California
CA_df = df[df['State'] == 'CA']

CA_df.head()

In [ ]:
#Zoom in on California
CA_df = df[df['State'] == 'CA']

CA_df.head()

In [ ]:
#plot all CA fires according to severity
(ggplot()
  + geom_map(california, fill= 'whitesmoke', color = 'grey')
  + geom_map(CA_df, aes(geometry = 'geometry', fill = 'dNBR_offst'), color = 'grey')
  + scale_fill_gradientn(colors = ["#beedcd", "#ed550e"],limits = [0, 200])
  + labs(fill='dNBR_offst')
  + theme_classic()
  + theme(axis_line=element_line(color="white"),
          axis_ticks=element_line(color = "white"),
          axis_text=element_line(color='white'),
          text=element_text(size = 12))
  # + xlab("")
  # + ylab("")
 )

In [ ]:
#Define animation constants
OUTPUT_DIR = "frames" #create directory to hold frames
CA_GIF = "map_animation.gif" #create final output file
FIG_WIDTH = 10 #set standard size for each animation frame
FIG_HEIGHT = 8 #set standard size for each animation frame
DPI = 100 #set standard resolution
FPS = 1 #set frames per second

In [ ]:
#define years for animation
years = sorted(CA_df['Year'].unique())

In [ ]:
#create CA frame boundary
min_x, min_y, max_x, max_y = california.total_bounds

#create 5% buffer around frame
buffer_x = (max_x - min_x) * 0.05
buffer_y = (max_y - min_y) * 0.05

#extend CA frame boundary using buffer
min_x -= buffer_x
min_y -= buffer_y
max_x += buffer_x
max_y += buffer_y

In [ ]:
#create a png for each year

frame_paths = [] #create empty frames to fill

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

for year in years: #create frame for individual year
    # Create filename
    frame_path = os.path.join(OUTPUT_DIR, f"frame_{year}.png")
    frame_paths.append(frame_path)

    # Generate plot for year
    fig, ax = plt.subplots(figsize=(FIG_WIDTH, FIG_HEIGHT))
    year_df = CA_df[CA_df['Year'] == year]

    # Add california
    california.boundary.plot(ax=ax, color='black', linewidth=1) #plot boundary
    california.plot(ax=ax, color='lightgray', alpha=0.5)  # Fill with light gray

    # Then plot the data if there is any for this year
    if not year_df.empty:
        year_df.plot(column='dNBR_offst', ax=ax,
                    cmap=plt.cm.RdYlGn_r,
                    vmin=0, vmax=200,
                    edgecolor='grey',
                    legend=True)

    ax.set_xlim(min_x, max_x) #set defined axis limits with buffer
    ax.set_ylim(min_y, max_y) #set defined axis limits with buffer

    ax.set_title(f'Year: {year}', fontsize=14)      # Set dynamic title
    ax.axis('off')                                  # Remove axes
    plt.tight_layout()

    # Save to file
    plt.savefig(frame_path, dpi=DPI)
    plt.close(fig)

In [ ]:
#combine yearly pngs into gif using imageio

images = []
for path in frame_paths:
    images.append(imageio.imread(path))

imageio.mimsave(CA_GIF, images, fps=FPS) #fill CA_GIF with set of images

display(Image(filename=CA_GIF))

#Upload State County Polygons

In [ ]:
#load in state boundary data
! unzip /content/drive/MyDrive/cb_2018_us_county_500k.zip #unpacks zipped folder

In [ ]:
shp = "cb_2018_us_county_500k.shp"
df_counties = gpd.read_file(shp) #reads in shape file
df_counties.head()

In [ ]:
#plot spatial data

(ggplot()
  + geom_map(df_counties, aes(geometry = 'geometry', fill='STATEFP'), show_legend = False)
  + labs(fill='STATEFP')
  + xlim(-5e6,3e6)
  + ylim(-2.5e6, 4e6)
  + theme_classic()
  + theme(axis_line=element_line(color="white"),
          axis_ticks=element_line(color = "white"),
          axis_text=element_line(color='white'),
          text=element_text(size = 12))
  # + xlab("")
  # + ylab("")
 )

#Initial Zoom in on State Data

In [ ]:
CA_df = df[df['State'] == 'CA']

CA_df.head()

In [ ]:
CA_df["dNBR_offst"].max()

In [ ]:
#plot spatial data

(ggplot()
  + geom_map(CA_df, aes(geometry = 'geometry', fill = 'dNBR_offst'), color = 'grey')
  + scale_fill_gradientn(colors = ["#beedcd", "#ed550e"],limits = [0, 200])
  + labs(fill='dNBR_offst')
  + theme_classic()
  + theme(axis_line=element_line(color="white"),
          axis_ticks=element_line(color = "white"),
          axis_text=element_line(color='white'),
          text=element_text(size = 12))
  # + xlab("")
  # + ylab("")
 )

In [ ]:
#plot spatial data

(ggplot()
  + geom_map(df, aes(geometry = 'geometry', fill='High_T'))
  + labs(fill='High_T')
  + theme_classic()
  + theme(axis_line=element_line(color="white"),
          axis_ticks=element_line(color = "white"),
          axis_text=element_line(color='white'),
          text=element_text(size = 12))
  # + xlab("")
  # + ylab("")
 )